# Amazon Stock: Technical Analysis 
### Candlestick, MACD, RSI, Bollinger Band, Stochastic Oscillator, Parabolic SAR & Yearly Growth

In [1]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.style.use('seaborn')
sns.set(font_scale=2)
import warnings; warnings.filterwarnings('ignore')

In [2]:
name='Amazon'
data=pd.read_csv('../input/amazonstockprice/Amazon.csv')
data['Date'] = pd.to_datetime(data['Date'])
ds_df=data[-300:].reset_index(drop=True)
ds_df2=data

In [3]:
display(ds_df)

,Date,High,Low,Open,Close,Volume,Adj Close
0,2021-01-19,3145.000000,3096.000000,3107.000000,3120.760010,3305100.0,3120.760010
1,2021-01-20,3279.800049,3175.000000,3181.989990,3263.379883,5309800.0,3263.379883
2,2021-01-21,3348.550049,3289.570068,3293.000000,3306.989990,4936100.0,3306.989990
3,2021-01-22,3321.909912,3283.159912,3304.310059,3292.229980,2821900.0,3292.229980
4,2021-01-25,3363.889893,3243.149902,3328.500000,3294.000000,3749800.0,3294.000000
...,...,...,...,...,...,...,...
295,2022-03-21,3261.679932,3191.060059,3222.419922,3229.830078,3326900.0,3229.830078
296,2022-03-22,3323.340088,3233.979980,3236.110107,3297.780029,3204300.0,3297.780029
297,2022-03-23,3327.399902,3253.739990,3274.100098,3268.159912,2790600.0,3268.159912
298,2022-03-24,3282.370117,3201.000000,3274.989990,3272.989990,2834900.0,3272.989990


# Candlestick

In [4]:
#fig=make_subplots(specs=[[{"secondary_y":True}]])
fig = go.Figure(go.Candlestick(x=ds_df['Date'],open=ds_df['Open'],high=ds_df['High'],low=ds_df['Low'],close=ds_df['Close']))
#fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['Close'],mode='lines',name='Close'))
fig.update_layout(title='Candlestick', yaxis_title='USD',width=800,height=500,)
fig.update_yaxes()
fig.show()

# Exponential Moving Average

In [5]:
ewma = pd.Series.ewm
ds_df['rolling_ema_12'] = ds_df.Close.ewm(min_periods=12, span=12).mean()
ds_df['rolling_ema_26'] = ds_df.Close.ewm(min_periods=26, span=26).mean()

In [6]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
#fig = go.Figure(go.Candlestick(x=ds_df['Date'],open=ds_df['Open'],high=ds_df['High'],low=ds_df['Low'],close=ds_df['Close']))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['rolling_ema_12'], mode='lines', name='EMA_12'))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['rolling_ema_26'], mode='lines', name='EMA_26'))
fig.update_layout(title='Exponential Moving Average', yaxis_title='USD',width=800,height=500,)
fig.update_yaxes()
fig.show()

# MACD

In [7]:
ds_df['close_12EMA'] = ewma(ds_df["Close"], span=12).mean()
ds_df['close_26EMA'] = ewma(ds_df["Close"], span=26).mean()
ds_df['MACD'] = ds_df['close_12EMA'] - ds_df['close_26EMA']
ds_df['MACD_Signal'] = ewma(ds_df["MACD"], span=9).mean()

In [8]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'),secondary_y=True,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['MACD'], mode='lines', name='MACD'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['MACD_Signal'], mode='lines', name='MACD_Signal'),secondary_y=False,)
fig.update_layout(autosize=False,width=800,height=400,title_text="MACD")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="MACD",secondary_y=False)
fig.update_yaxes(title_text="USD",secondary_y=True)
fig.show()

In [9]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'),secondary_y=True,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['MACD']-ds_df['MACD_Signal'], mode='lines', name='MACD - MACD_Signal'),secondary_y=False,)
fig.update_layout(autosize=False,width=800,height=400,title_text="MACD - MACD_Signal")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="MACD",secondary_y=False)
fig.update_yaxes(title_text="USD",secondary_y=True)
fig.show()

# RSI

In [10]:
def rsiFunc(prices, n=14):
    
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] 
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [11]:
ds_df['rsi_6'] = rsiFunc(ds_df['Close'].values, 6)
ds_df['rsi_14'] = rsiFunc(ds_df['Close'].values, 14)
ds_df['rsi_20'] = rsiFunc(ds_df['Close'].values, 20)

In [12]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.update_layout(title='RSI', yaxis_title='USD',width=800,height=400,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'), secondary_y=True,)
#fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'rsi_6'], mode='lines', name=f'rsi_6'),secondary_y=False)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'rsi_14'], mode='lines', name=f'rsi_14'),secondary_y=False)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'rsi_20'], mode='lines', name=f'rsi_20'),secondary_y=False)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="RSI",secondary_y=False)
fig.update_yaxes(title_text="USD",secondary_y=True)
fig.show()

# Bollinger Band

In [13]:
window = 20
no_of_std = 2

ds_df[f'MA_{window}MA'] = ds_df['Close'].rolling(window=window).mean()
ds_df[f'MA_{window}MA_std'] = ds_df['Close'].rolling(window=window).std() 
ds_df[f'MA_{window}MA_BB_high'] = ds_df[f'MA_{window}MA'] + no_of_std * ds_df[f'MA_{window}MA_std']
ds_df[f'MA_{window}MA_BB_low'] = ds_df[f'MA_{window}MA'] - no_of_std * ds_df[f'MA_{window}MA_std']

In [14]:
fig = go.Figure()
fig.update_layout(title='Bollinger Band', yaxis_title='USD',width=800,height=400,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA'], mode='lines', name=f'BB_mean'))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA_BB_high'], mode='lines', name=f'BB_high'))
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA_BB_low'], mode='lines', name=f'BB_low'))
fig.show()

# Stochastic Oscillator

In [15]:
ds_df['Low_min']=ds_df['Low'].rolling(window=14,center=False).min()
ds_df['High_max']=ds_df['High'].rolling(window=14,center=False).max()
ds_df['STOK'] = ((ds_df['Close']-ds_df['Low_min'])/(ds_df['High_max']-ds_df['Low_min']))*100
ds_df['STOD'] = ds_df['STOK'].rolling(window=3,center=False).mean()

In [16]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['STOK'],mode='lines',name='STOK'),secondary_y=True,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['STOD'],mode='lines',name='STOD'),secondary_y=True,)
fig.update_layout(title='Stochastic Oscillator', yaxis_title='USD',width=800,height=400,)
fig.show()

# Parabolic SAR

In [17]:
def psar(df, iaf = 0.02, maxaf = 0.2):
    length = len(df)
    dates = (df['Date'])
    high = (df['High'])
    low = (df['Low'])
    close = (df['Close'])
    psar = df['Close'][0:len(df['Close'])]
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    ep = df['Low'][0]
    hp = df['High'][0]
    lp = df['Low'][0]
    for i in range(2,length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if df['Low'][i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = df['Low'][i]
                af = iaf
        else:
            if df['High'][i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = df['High'][i]
                af = iaf
        if not reverse:
            if bull:
                if df['High'][i] > hp:
                    hp = df['High'][i]
                    af = min(af + iaf, maxaf)
                if df['Low'][i - 1] < psar[i]:
                    psar[i] = df['Low'][i - 1]
                if df['Low'][i - 2] < psar[i]:
                    psar[i] = df['Low'][i - 2]
            else:
                if df['Low'][i] < lp:
                    lp = df['Low'][i]
                    af = min(af + iaf, maxaf)
                if df['High'][i - 1] > psar[i]:
                    psar[i] = df['High'][i - 1]
                if df['High'][i - 2] > psar[i]:
                    psar[i] = df['High'][i - 2]
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]
    #return {"dates":dates, "high":high, "low":low, "close":close, "psar":psar, "psarbear":psarbear, "psarbull":psarbull}
    #return psar, psarbear, psarbull

    df['psar'] = psar
    df['psarbear'] = psarbear
    df['psarbull'] = psarbull

In [18]:
psar(ds_df)

In [19]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
#fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['Close'],mode='lines',name='Close'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['psar'],mode='lines',name='psar'),secondary_y=True,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['psarbear'],mode='lines',name='psarbear'),secondary_y=True,)
fig.add_trace(go.Scatter(x=ds_df['Date'], y=ds_df['psarbull'],mode='lines',name='psarbull'),secondary_y=True,)
fig.update_layout(title='Parabolic SAR', yaxis_title='USD',width=800,height=400,)
fig.show()

# Ichimoku Clouds

In [20]:
# need add future dates
from datetime import datetime
from datetime import timedelta
ds_df['Date']=ds_df['Date'].astype(str)
latest0 = datetime.strptime(ds_df['Date'].max(),'%Y-%m-%d').date()
dates0=[]
for i in range(1,40):
    dates0+=[(latest0+timedelta(i)).strftime('%Y-%m-%d')]
ADD=pd.DataFrame(columns=ds_df.columns)
ADD['Date']=dates0
ds_df=pd.concat([ds_df,ADD],axis=0).reset_index(drop=True)

In [21]:
ds_df['base9']=(ds_df['High'].rolling(window=9).max()+ds_df['High'].rolling(window=9).min())/2
ds_df['base26']=(ds_df['High'].rolling(window=26).max()+ds_df['High'].rolling(window=26).min())/2
ds_df['base52']=(ds_df['High'].rolling(window=52).max()+ds_df['High'].rolling(window=52).min())/2
ds_df['leading span1']=(ds_df['base9']/2+ds_df['base52']/2).shift(26)
ds_df['leading span2']=ds_df['base52'].shift(26)
ds_df['lagging span']=ds_df['Close'].shift(-26)

In [22]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['Close'],name='Close'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['base9'],name='base9'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['base26'],name='base26'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['leading span1'],name='leading span1'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['leading span2'],name='leading span2'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['lagging span'],name='lagging span'),secondary_y=False,)
fig.update_layout(autosize=False,width=800,height=400,title_text='Ichimoku Clouds')
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="USD",secondary_y=False)
fig.update_yaxes(title_text="USD",secondary_y=True)
fig.show()

#### The area surrounded by leading span1 and leading span2 is Ichimoku clouds.

# Yearly Growth (original)

In [23]:
ds_df2['Close MA20'] = ds_df2['Close'].rolling(window=20).mean()
ds_df2['Close MA20 shift year']=ds_df2['Close MA20'].shift(253)
ds_df2['Yearly Growth']=(ds_df2['Close MA20']-ds_df2['Close MA20 shift year'])*100/ds_df2['Close MA20 shift year']

In [24]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=ds_df2['Date'][500:], y=ds_df2['Close MA20'][500:],mode='lines',name='Close MA20'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df2['Date'][500:], y=ds_df2['Yearly Growth'][500:],mode='lines',name='Yearly Growth'),secondary_y=True,)
fig.update_layout(title='Close MA20 and Yearly Growth',width=800,height=400,)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Close MA20 USD",secondary_y=False)
fig.update_yaxes(title_text="Yearly Growth %",secondary_y=True)
fig.show()